In [1]:
import pandas as pd

In [2]:
sap_df = pd.read_csv('exports/beli_retur_sap.csv')
sc_df = pd.read_csv('exports/beli_retur_sc.csv')

In [3]:
sap_df["posting_date"] = pd.to_datetime(sap_df["posting_date"])

sap_df["outlet"] = sap_df["profit_center"]
sc_df["outlet"] = sc_df["kode_outlet"]

sap_df["bulan"] = sap_df["posting_date"].dt.month
sap_df.drop(columns=["posting_date"], inplace=True)

sc_df['jml_retur'] = -sc_df['jml_retur'].abs()
sap_grouped = sap_df.groupby(["outlet", "bulan"], as_index=False)["kredit"].sum()
sc_grouped = sc_df.groupby(["outlet", "bulan"], as_index=False)["jml_retur"].sum()

In [8]:
sap_grouped

,outlet,bulan,kredit
0,BX01,1,440757090
1,BX01,2,476115649
2,BX01,3,542067357
3,BX01,4,253739055
4,BX01,5,406737961
...,...,...,...
179,BX52,1,115707419
180,BX52,2,126931009
181,BX52,3,122112575
182,BX52,4,72584048


In [7]:
validasi = pd.merge(sap_grouped, sc_grouped, on=["outlet", "bulan"], how="outer")
validasi["selisih"] = validasi["kredit"] - validasi["jml_retur"]
validasi["status"] = validasi["selisih"].apply(lambda x: "VALID" if abs(x) < 1 else "TIDAK VALID")

validasi.sort_values(by=["status", "outlet", "bulan"], ascending=[False, True, True])


,outlet,bulan,kredit,jml_retur,selisih,status
17,BX05,4,-25998000.0,-25998000.0,0.0,VALID
60,BX18,4,-27688495.0,-27688495.0,0.0,VALID
102,BX28,1,-338404.0,-338404.0,0.0,VALID
104,BX28,4,-300000.0,-300000.0,0.0,VALID
136,BX36,2,-24502.0,-24502.0,0.0,VALID
...,...,...,...,...,...,...
183,BX52,1,115707419.0,-2424481.0,118131900.0,TIDAK VALID
184,BX52,2,126931009.0,-6284880.0,133215889.0,TIDAK VALID
185,BX52,3,122112575.0,-6558407.0,128670982.0,TIDAK VALID
186,BX52,4,72584048.0,-3087439.0,75671487.0,TIDAK VALID
